In [1]:
import os

import pandas as pd
import numpy as np

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

from flask import Flask, jsonify, render_template
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)

In [2]:
app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///db/bellybutton.sqlite"
db = SQLAlchemy(app)

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(db.engine, reflect=True)

# Save references to each table
Samples_Metadata = Base.classes.sample_metadata
Samples = Base.classes.samples

C:\Users\Stefan\Anaconda3\lib\site-packages\flask_sqlalchemy\__init__.py:835: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '


In [10]:
    # Use Pandas to perform the sql query
stmt = db.session.query(Samples).statement
df = pd.read_sql_query(stmt, db.session.bind)

    # Return a list of the column names (sample names)
print(df)

      otu_id                                          otu_label  940  941  \
0          1  Archaea;Euryarchaeota;Halobacteria;Halobacteri...    0    0   
1          2  Archaea;Euryarchaeota;Halobacteria;Halobacteri...    0    0   
2          3  Archaea;Euryarchaeota;Halobacteria;Halobacteri...    0    0   
3          4  Archaea;Euryarchaeota;Methanobacteria;Methanob...    0    0   
4          5  Archaea;Euryarchaeota;Methanobacteria;Methanob...    0    0   
5          6  Archaea;Euryarchaeota;Methanobacteria;Methanob...    0    0   
6          7                                           Bacteria    0    0   
7          8                                           Bacteria    0    0   
8          9                                           Bacteria    0    0   
9         10                                           Bacteria    0    0   
10        11                                           Bacteria    1    0   
11        12                                           Bacteria    0    0   

In [20]:
def sample_metadata(sample):
    sel = [
        Samples_Metadata.sample,
        Samples_Metadata.ETHNICITY,
        Samples_Metadata.GENDER,
        Samples_Metadata.AGE,
        Samples_Metadata.LOCATION,
        Samples_Metadata.BBTYPE,
        Samples_Metadata.WFREQ,
        ]
    
    results = db.session.query(*sel).filter(Samples_Metadata.sample == sample).all()
       # Create a dictionary entry for each row of metadata information
    sample_metadata = {}
    for result in results:
        sample_metadata["sample"] = result[0]
        sample_metadata["ETHNICITY"] = result[1]
        sample_metadata["GENDER"] = result[2]
        sample_metadata["AGE"] = result[3]
        sample_metadata["LOCATION"] = result[4]
        sample_metadata["BBTYPE"] = result[5]
        sample_metadata["WFREQ"] = result[6]

    return(sample_metadata)
    return(jsonify(sample_metadata))


In [22]:
sample_metadata()

TypeError: sample_metadata() missing 1 required positional argument: 'sample'

NameError: name 'results' is not defined

In [ ]:
@app.route("/samples/<sample>")
def samples(sample):
    """Return `otu_ids`, `otu_labels`,and `sample_values`."""
    stmt = db.session.query(Samples).statement
    df = pd.read_sql_query(stmt, db.session.bind)

    # Filter the data based on the sample number and
    # only keep rows with values above 1
    sample_data = df.loc[df[sample] > 1, ["otu_id", "otu_label", sample]]

    # Sort by sample
    sample_data.sort_values(by=sample, ascending=False, inplace=True)

    # Format the data to send as json
    data = {
        "otu_ids": sample_data.otu_id.values.tolist(),
        "sample_values": sample_data[sample].values.tolist(),
        "otu_labels": sample_data.otu_label.tolist(),
    }
    return jsonify(data)
